## 1.先把csv里的数据读取出来

这里要注意一点就是，需要sys.path.append添加项目根路径，<br>
不然的话，python执行会找不到config和tools这两个自定义<br>
模块，而且需要写在import之前！

In [1]:
import requests as rq
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
import config as c
import tools as t

result_save_file = c.outputs_logs_path + 'wzly.csv'
raw_data = pd.read_csv(result_save_file)

## 2.分析身高情况

这里要对数据做下清洗，流程如下：

- 1.筛选140-200以内的身高值
- 2.把身高按照升序排列
- 3.按照三个身高段，对数据进行分段处理，依次是140,150,160,170,180

In [5]:
from pyecharts import Bar


# 身高范围
height_interval = ['140cm', '150cm', '160cm', '170cm', '180cm'] 

# 分析身高
def analysis_height(data):
    height_data = data['身高']
    height = (height_data.loc[(height_data > 140) & (height_data < 200)]).value_counts().sort_index()
    height_count = [0, 0, 0, 0, 0]
    for h in range(0, len(height)):
        if 140 <= height.index[h] < 150:
            height_count[0] += height.values[h]
        elif 150 <= height.index[h] < 160:
            height_count[1] += height.values[h]
        elif 160 <= height.index[h] < 170: 
            height_count[2] += height.values[h]
        elif 170 <= height.index[h] < 180:
            height_count[3] += height.values[h]
        elif 180 <= height.index[h] < 190:
            height_count[4] += height.values[h]
    return height_count

# 绘制身高分布柱状图
def draw_height_bar(data):
    bar = Bar("妹子身高分布柱状图")
    bar.add("妹子身高",height_interval,data,bar_category_gap=0,is_random=True,)
    return bar
    

draw_height_bar(analysis_height(raw_data))

In [12]:
from pyecharts import Pie


# 绘制身高分布饼图
def draw_height_pie(data):
    pie = Pie("妹子身高分布饼图-圆环图",title_pos='center')
    pie.add("", height_interval, data, radius=[40, 75], label_text_color=None,
        is_label_show=True, legend_orient='vertical',is_random=True,
        legend_pos='left')
    return pie

draw_height_pie(analysis_height(raw_data))

**分析时间**：

从上面两个图可以看出，占比最高的身高范围是：**160-170cm**，占比达**68.13%**；<br>
其次是 **150-160cm**，占比 **26.08%**，170-180cm的占比5.61%，(¯﹃¯)啧啧，<br>
像我这种勉强到170cm的应该还是有比较多机会的。

---

## 3.分析学历情况

接着来分析一波学历情况

In [13]:
from pyecharts import Funnel


edu_interval = ['本科', '大专', '高中', '中专', '初中', '硕士', '博士', '院士']  # 学历范围

# 分析学历
def analysis_edu(data):
    return data['学历'].value_counts().values


# 学历漏斗图
def draw_edu_funnel(data):
    funnel = Funnel("妹子学历分布漏斗图",title_top='center')
    funnel.add("学历", edu_interval, data, is_label_show=True,label_pos="inside", label_text_color="#fff",is_random=True )
    return funnel

draw_edu_funnel(analysis_edu(raw_data))

**分析时间**：

从上面的漏斗图看出，交友的大部分妹子大部分都是本科与大专的，高中，中专和初中其次，<br>
最令我好奇的是竟然有4个院士？？？怀着好奇心我搜了下csv里对应的数据：<br>
一个是良缘小秘书，我主良缘官方帐号；<br>接着有两个：<br>
http://www.7799520.com/user/616763.html<br>
http://www.7799520.com/user/554309.html<br>
最后还有一个：<br>
http://www.7799520.com/user/418246.html<br>
打开一看:<br>
![](http://static.zybuluo.com/coder-pig/5czocb1cplikn7achj89xu8d/1.png)
然后官网还写着这样的骚话：
![](http://static.zybuluo.com/coder-pig/d8ijdz40wg5tr6uk4ul26dzr/2.png)
我TM...
![](http://static.zybuluo.com/coder-pig/dtdec6xdj1770it2s4axwdmc/3.png)

---

## 4.分析年龄情况

接着来分析以下都是什么年龄段的妹子找对象(什么年龄段的妹子比较着急...)<br>
吐槽下年龄这个，填资料的人不是一般的皮：<br>

![](http://static.zybuluo.com/coder-pig/2iqvey783t2a923ea4xgcdx6/1.png)

![](http://static.zybuluo.com/coder-pig/3yaev1s475eeo4kwzl61c5oe/2.png)

<br>
这些垃圾值肯定是要过滤一波的，所有筛选的年龄段范围为：1956-2000，<br>
分为五个年龄段：18-25,25-30,31-40,41-50和50以上。<br>
接着写代码过滤下统计一波！<br>

In [12]:
from pyecharts import Radar


# 学历范围
age_interval = [
    ('18-25', 8000), ('26-30', 8000), ('31-40', 8000),
    ('41-50', 8000), ('50以上', 8000),
] 

# 分析年龄
def analysis_age(data):
    age_data = data['出生年份']
    age = (age_data.loc[(age_data >= 1956) & (age_data <= 2000)]).value_counts().sort_index()
    age_count = [0, 0, 0, 0, 0]
    for h in range(0, len(age)):
        if 1993 <= age.index[h] <= 2000:
            age_count[0] += age.values[h]
        elif 1988 <= age.index[h] <= 1992:
            age_count[1] += age.values[h]
        elif 1978 <= age.index[h] <= 1987:
            age_count[2] += age.values[h]
        elif 1968 <= age.index[h] <= 1977:
            age_count[3] += age.values[h]
        elif age.index[h] < 1968:
            age_count[4] += age.values[h]
    return age_count

# 年龄雷达图
def draw_age_radar(data):
    radar = Radar("妹子年龄分布雷达图")
    radar.config(age_interval)
    radar.add("年龄段", data, is_area_show=False,legend_selectedmode='single')
    return radar

draw_age_radar([analysis_age(raw_data)])

**分析时间**：

从雷达图上面可以看出，26-30比较热衷与找对象脱单，人数达到了**7355人**，<br>
接着是18-25的，有**4313人**之多，正常，女的都结婚比较早，想想小学同学<br>
的儿子都上小学了，我还单着，**哭哭/(ㄒoㄒ)/~~ **。接着是31-40岁的大龄剩女，<br>
也有3417人那么多，都是到了年纪才着急么，恐怖...50岁以上的则有77人。

---

## 5.分析下城市

接下来分析下找对象妹子的城市分布

In [13]:
from pyecharts import Geo


# 分析城市分布
def analysis_city(data):
    city_data = data['城市'].value_counts()
    city_list = []
    for city in range(0, len(city_data)):
        if city_data.values[city] > 10:
            city_list.append((city_data.index[city], city_data.values[city]))
    return city_list

# 城市分布地图
def draw_city_geo(data):
    geo = Geo("全国妹子所在地分布", "制作人：CoderPIg", title_color="#fff",
              title_pos="left", width=1200,
              height=600, background_color='#404a59')
    attr, value = geo.cast(data)
    geo.add("", attr, value, visual_range=[10, 2500],  visual_text_color="#fff",
            symbol_size=15, is_visualmap=True)
    return geo

draw_city_geo(analysis_city(raw_data))

**分析时间:**<br>
<br>
左侧滑动条是可以滑动的，用于设置人数范围，把low游标往上拉，滚动滚动可缩放<br>
![](http://static.zybuluo.com/coder-pig/zzaw5rvfywjpdkxrtlwhby8l/1.png)
<br>
从下往上依次是(可以把鼠标移到点上查看具体情况)：<br>
<br>
**广州2469，深圳2372，上海2319，北京1231**<br>
<br>
low游标往下拉一点，拉到250-1000左右，又有几个新的点出现：<br>
<br>
**长沙802，杭州751，南京655，厦门651，福州611，成都532，武汉425，青岛308**<br>
<br>
再拉，99-250左右，有三个新的点：<br>
<br>
**山东230，苏州161，重庆126**<br>
<br>
剩下的点比较零碎就不分析了，单身妹子所在地大部分还是集中在北上广深，<br>
第二梯队的长沙，杭州，南京，厦门，福州，成都，武汉，青岛...

## 6. 词频分析

接下来分析一波交友宣言，看看妹子们都喜欢怎么样的男生。<br>
需要先过滤一波系统默认的交友宣言，一般自己写的都是不会<br>
重复的，这个利用pandas.value_counts()，只保留出现一次的<br>
交友宣言，然后利用Jieba分词，Counter统计词频，剔除标点和特殊符号，<br>
同时过滤掉一些无意义的无用词，取前100个关键词，<br>
绘制成词云，以此了解妹子们的喜好。<br>

In [14]:
from pyecharts import WordCloud
import re
import jieba as jb
from collections import Counter


# 过滤标点符号正则
word_pattern = re.compile('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？“”、~@#￥%……&*（）(\d+)]+')
# 过滤无用词
exclude_words = [
            '一辈子', '不相离', '另一半', '业余时间', '性格特点', '茫茫人海', '男朋友', '找对象',
            '谈恋爱', '有时候', '女孩子', '哈哈哈', '加微信', '兴趣爱好',
            '是因为', '不良嗜好', '男孩子', '为什么', '没关系', '不介意',
            '没什么', '交朋友', '大大咧咧', '大富大贵', '联系方式', '打招呼',
            '有意者', '晚一点', '哈哈哈', '以上学历', '是不是', '给我发',
            '不怎么', '第一次', '越来越', '遇一人', '择一人', '无数次',
            '符合条件', '什么样', '全世界', '比较简单', '浪费时间', '不知不觉',
            '有没有', '寻寻觅觅', '自我介绍', '请勿打扰', '差不多', '不在乎', '看起来',
            '一点点', '陪你到', '这么久', '看清楚', '身高体重', '比较慢', '比较忙',
            '多一点', '小女生', '土生土长', '发消息', '最合适'
        ]

# 词频分布
def analysis_word(data):
    word_data = data['交友宣言'].value_counts()
    word_list = []
    for word in range(0, len(word_data)):
        if word_data.values[word] == 1:
            word_list.append(word_data.index[word])
    return word_list

# 交友宣言词云
def draw_word_wc(name, count):
    wc = WordCloud(width=900, height=720)
    wc.add("", name, count, word_size_range=[20, 100], shape='diamond')
    return wc

word_result = word_pattern.sub("", ''.join(analysis_word(raw_data)))
# Jieba分词
words = [word for word in jb.cut(word_result, cut_all=False) if len(word) >= 3]
# 遍历过滤无用词
for i in range(0, len(words)):
    if words[i] in exclude_words:
        words[i] = None
filter_list = list(filter(lambda t: t is not None, words))
data = r' '.join(filter_list)
# 词频统计
c = Counter(filter_list)
word_name = []  # 词
word_count = []  # 词频
for word_freq in c.most_common(100):
    word, freq = word_freq
    word_name.append(word)
    word_count.append(freq)
draw_word_wc(word_name, word_count)    

**分析时间：**<br>
<br>
排名前八的关键词：**责任心，上进心，事业心，热爱生活，性格开朗，脾气好，孝顺父母，安全感**<br>
除此之外，我觉得两个人在一起更重要的是**三观差不多，包容心，相互理解**，以及**基本的**<br>
**经济基础**吧，毕竟贫贱夫妻百事哀，每天都在为油盐酱醋奔波，天天操心一些鸡毛蒜皮的事，<br>
何来的快乐？<br>